In [1]:
 import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
52,47,Female,100.0,Strong,Bangalore
14,51,Male,104.0,Mild,Bangalore
51,11,Female,100.0,Strong,Kolkata
46,19,Female,101.0,Mild,Mumbai
28,16,Male,104.0,Mild,Kolkata
...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore
5,84,Female,NaN,Mild,Bangalore
58,23,Male,98.0,Strong,Mumbai
61,81,Female,98.0,Strong,Mumbai


In [8]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [9]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

### 1. Aam Zindigi

In [11]:
# adding simple imputer to fever col to fill up missing values
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

###also add the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [19]:
from sklearn.preprocessing import OrdinalEncoder

# Ordinal encoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']]) # I have basically said mild ko chota value and strong ko bada value
X_train_cough = oe.fit_transform(X_train[['cough']])

# Test data encoding using the fitted encoder 
X_test_cough = oe.transform(X_test[['cough']])

print(X_train_cough.shape) 


(80, 1)


In [21]:
# OneHotEncoding -> Gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also for the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [22]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also for the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

### Now i have all of the columns so now i will merge them all

In [26]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)
X_train_transformed.shape

(80, 7)

Ye sirf 4 columns the so it took alot of time there is an alternative approach which is better

### Mentos Zindagi

In [27]:
from sklearn.compose import ColumnTransformer

In [39]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

1. I am importing the **ColumnTransformer** class from the **sklearn.compose** module. This class allows you to apply different preprocessing techniques to specific columns within your dataset.
2. Creating the ColumnTransformer object
    1. Piece of code
    * some_transformer = ColumnTransformer(
        * transformers=[
            * ('tnf1', SimpleImputer(), ['Fever']) 
            * ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['Cough']),
            * ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['Gender','City'])
        * ],remainder='passthrough')
creating an instance of the ColumnTransformer class and giving it an arbitrary name (**some_transformer** in this example).
Key parameters in the code:
    * ('tnf1', SimpleImputer(), ['Fever']):
        * Name: 'tnf1'
        * Transformer: SimpleImputer – Handles missing values by imputing them (likely with the mean).
        * Columns: Applied to the 'Fever' column.
    * ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['Cough'])
        * Name: 'tnf2'
        * Transformer: OrdinalEncoder – Encodes categorical features with an ordinal relationship (e.g., 'Mild' and 'Strong' imply an order).
        * Columns: Applied to the 'Cough' column.
    * ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['Gender', 'City'])
        * Name: 'tnf3'
        * Transformer: OneHotEncoder – Creates new binary columns for each unique category within a column. To avoid redundancy, one category is * dropped (drop='first').
        * Columns: Applied to the 'Gender' and 'City' columns.

remainder='passthrough': This instructs the ColumnTransformer to include all columns from your original dataset that are not explicitly listed within the transformers. These columns will be passed through without any modification.

In [40]:
transformer.fit_transform(X_train)

array([[100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  51.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  11.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  19.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  16.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.    

In [41]:
transformer.fit_transform(X_train).shape


(80, 7)

In [42]:
transformer.transform(X_test).shape

(20, 7)